# A simple chi analysis

In this example we will grab some data for a simple chi analysis. First, import the relevent modules. 

In [1]:
import lsdviztools.lsdbasemaptools as bmt
from lsdviztools.lsdplottingtools import lsdmap_gdalio as gio
import lsdviztools.lsdmapwrappers as lsdmw
import lsdttparamselector as ps

## Part 1: Grab some data

We will grab some data from the Shillong Plateau in India. I use google maps to get the latitude and longitude of the southwest and northeast corners. Once you give the `ot_scraper` the corners and the type of data to download (in this case SRTM30 data), you can move on to downloading.

In [3]:
SH_DEM = bmt.ot_scraper(source = "SRTM30",longitude_W = 91.271109, longitude_E = 92.214265, latitude_S = 25.107807, latitude_N = 25.627673,prefix = "shillong")
SH_DEM.print_parameters()

The source is: SRTM30
The west longitude is: 90.795518
The east longitude is: 92.347537
The south latitude is: 25.763649
The north latitude is: 25.107807
The path is: ./
The prefix is: shillong


Now call the download. You need to wait until the `*` symbol dissappears from the right of the line (when it stops saying `In[*]`), that indicates the download has finished. 

In [5]:
SH_DEM.download_pythonic()

I am going to download the following for you:
https://portal.opentopography.org/otr/getdem?demtype=SRTMGL1&south=25.107807&north=25.763649&west=90.795518&east=92.347537&outputFormat=GTiff
This might take a little while, depending on the size of the file. 
Finished downloading


Okay, now we need to do two things. First, we need to convert to UTM. Then we need to convert to ENVI bil format. 

In [6]:
SH_DEM.to_UTM_pythonic()
RasterFile = "shillong_SRTM30_UTM.tif"
DataDirectory = "./"
gio.convert2bil(DataDirectory, RasterFile,minimum_elevation=0)

The projections is:
GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]
And some extra projection information strings:
None
WGS 84
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 5587, 'height': 2361, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0002777777777778146, 0.0, 90.79541666670252,
       0.0, -0.0002777777777778146, 25.76374999999547)}
res tuple is:
(30.0, 30.0)
The destination CRS is: EPSG:32646
{'driver': 'GTiff', 'dtype': 'int16', 'nodata': -32768.0, 'width': 5229, 'height': 2478, 'count': 1, 'crs': CRS.from_epsg(32646), 'transform': Affine(30.0, 0.0, 277695.49411623424,
       0.0, -30.0, 2851370.6046270058)}
Removing pixels with an elevation less than 0


Okay, we now have an ENVI bil DEM. We are ready to start doing topographic analysis!

## Part 2, get some basin outlets

I want to extract some specific basins. I will get the lat-long of their outlets to select them (use *What's here* on google maps. I then create a csv containing the latitude and longitude using pandas. 

In [7]:
import pandas as pd

In [14]:
d = {'latitude': [25.187265,25.178305,25.176887,25.183027,25.197690], 'longitude': [92.016604,91.756626,91.633770,91.378109,91.255284]}
df = pd.DataFrame(data=d)
print(df)

    latitude  longitude
0  25.187265  92.016604
1  25.178305  91.756626
2  25.176887  91.633770
3  25.183027  91.378109
4  25.197690  91.255284


In [15]:
df.to_csv("basin_outlets.csv",index=False)

## Part 3: Running the analysis

In [19]:
lsdtt_parameters = {"write_hillshade" : "true", 
                    "get_basins_from_outlets" : "true", 
                    "basin_outlet_csv" : "basin_outlets.csv", 
                    "search_radius_nodes" : "15",
                    "convert_csv_to_geojson" : "true",
                    "print_basin_raster" : "true",
                    "print_chi_data_maps" : "true"}

In [20]:
lsdtt_drive = lsdmw.lsdtt_driver(read_prefix = "shillong_SRTM30_UTM",
                                 write_prefix= "shillong_SRTM30_UTM",
                                 parameter_dictionary=lsdtt_parameters)
lsdtt_drive.print_parameters()

The command line tool is: lsdtt-basic-metrics
The driver name is: Test_01
The read path is: ./
The write path is: ./
The read prefix is: shillong_SRTM30_UTM
The write prefix is: shillong_SRTM30_UTM
The parameter dictionary is:
{'write_hillshade': 'true', 'get_basins_from_outlets': 'true', 'basin_outlet_csv': 'basin_outlets.csv', 'search_radius_nodes': '15', 'convert_csv_to_geojson': 'true', 'print_basin_raster': 'true', 'print_chi_data_maps': 'true'}


In [21]:
lsdtt_drive.run_lsdtt_command_line_tool()

Done writing the driver file
I've finised writing the driver file. Let me run LSDTT for you.


## Part 3: Plotting the data

First, lets just see where we are.

In [22]:
DataDirectory = "./"
Base_file = "shillong_SRTM30_UTM"  
img_name = lsdmw.SimpleHillshade(DataDirectory,Base_file,cmap="gist_earth")

Your colourbar will be located: right
xsize: 5229 and y size: 2478
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 46N
WGS 84 / UTM zone 46N
This seems to be from the new gdal version
Zone string is: 46N
The zone is: 46
And the hemisphere is: N
The EPSG string is: epsg:32646
epsg:32646
minimum values are x: 277695.0 and y: 2777030.0
I made the ticks.
x labels are: 
['300', '330', '360', '390', '420']
x locations are:
[300000.0, 330000.0, 360000.0, 390000.0, 420000.0]
y labels are: 
['2800', '2830']
y locations are:
[2800000.0, 2830000.0]
This colourmap is: gray
The number of axes are: 1
Axes(0,0;1x1)
Axes(0,0;1x1)
N axes are: 1
Axes(0,0;1x1)
xsize: 5229 and y size: 2478
NoData is: -9999.0
Let me get that projection for you
In this function I will extract the UTM zone
The dataset is projected.
The projection string is: WGS 84 / UTM zone 46N
WGS 84 / UTM zone 46N
This 

findfont: Font family ['Liberation Sans'] not found. Falling back to DejaVu Sans.


In [23]:
img_name

'./shillong_SRTM30_UTM_hillshade.png'

<img src="shillong_SRTM30_UTM_hillshade.png" width=800>